In [ ]:
!python -m pip install findspark

In [ ]:
! pip install pyspark==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 MB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 10.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612222 sha256=ccf361063db3e970ceb019b7cfd1e242835387d9ce364c5d27cdeb24a51880d1
  Stored in directory: /root/.cache/pip/wheels/19/b0/c8/6cb894117070e130fc44352c2a13f15b6c27e440d04a84fb48
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.3
    Uninstalling pyspark-3.5.3:
      Successfully uninstalled pyspark-3.5.3


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
GTRDD = spark.sparkContext.textFile('GlobalTemperatures.csv')
GLTBCRDD = spark.sparkContext.textFile('GlobalLandTemperaturesByCountry.csv')
GLTBSRDD = spark.sparkContext.textFile('GlobalLandTemperaturesByState.csv')
GLTBMCRDD = spark.sparkContext.textFile('GlobalLandTemperaturesByMajorCity.csv')
GLTBCiRDD = spark.sparkContext.textFile('GlobalLandTemperaturesByCity.csv')

In [ ]:
# 1. For each month, show the name of states having the highest temperature in The US.
result1 = GLTBSRDD.map(lambda x: x.split(',')) \
    .filter(lambda x: x[4] == "United States" and x[0].startswith("2004")) \
    .map(lambda x: ((x[0][:7], x[3]), float(x[1]) if x[1] else float('-inf'))) \
    .reduceByKey(max) \
    .map(lambda x: (x[0][0], (x[0][1], x[1]))) \
    .groupByKey() \
    .mapValues(lambda vals: max(vals, key=lambda x: x[1])) \
    .sortByKey() \
    .collect()

for item in result1:
    print(f"{item[0]},{item[1][0]},{item[1][1]}")

2004-01,Hawaii,21.833000000000002
2004-02,Hawaii,22.377
2004-03,Hawaii,21.551
2004-04,Hawaii,21.858
2004-05,Florida,24.929000000000002
2004-06,Florida,27.709
2004-07,Florida,28.033
2004-08,Florida,27.793000000000006
2004-09,Florida,27.051
2004-10,Hawaii,24.596
2004-11,Hawaii,23.753
2004-12,Hawaii,22.335


In [ ]:
# 2. Determine the average temperature over the years of each country.
import statistics

result2 = GLTBCRDD.map(lambda x: x.split(',')) \
    .filter(lambda x: x[0].startswith("2004")) \
    .map(lambda x: ((x[0][:4], x[3]), float(x[1]) if x[1] else float('-inf'))) \
    .groupByKey() \
    .mapValues(lambda vals: statistics.mean(list(vals))) \
    .map(lambda x: (x[0][1], (x[0][0], x[1]))) \
    .groupByKey() \
    .mapValues(lambda vals: max(vals, key=lambda x: x[1])) \
    .sortByKey() \
    .collect()

for item in result2:
    print(f"{item[0]},{item[1]}")

"Bonaire,('2004', 27.69125)
Afghanistan,('2004', 15.770916666666666)
Africa,('2004', 24.895916666666665)
Albania,('2004', 13.258166666666666)
Algeria,('2004', 23.916)
American Samoa,('2004', 27.281166666666667)
Andorra,('2004', 12.043000000000001)
Angola,('2004', 22.572833333333335)
Anguilla,('2004', 27.334000000000003)
Antarctica,('2004', -inf)
Antigua And Barbuda,('2004', 27.15116666666667)
Argentina,('2004', 15.442166666666667)
Armenia,('2004', 9.720166666666666)
Aruba,('2004', 28.506)
Asia,('2004', 8.488833333333334)
Australia,('2004', 22.286916666666666)
Austria,('2004', 6.906083333333333)
Azerbaijan,('2004', 12.6195)
Bahamas,('2004', 25.84975)
Bahrain,('2004', 27.15025)
Baker Island,('2004', 26.07625)
Bangladesh,('2004', 25.386333333333333)
Barbados,('2004', 27.225416666666668)
Belarus,('2004', 6.905833333333334)
Belgium,('2004', 10.452583333333333)
Belize,('2004', 25.744083333333336)
Benin,('2004', 27.872)
Bhutan,('2004', 12.208083333333333)
Bolivia,('2004', 21.571)
Bosnia And H

In [ ]:
#3. Determine the cities of China which have the temperature lower than 0.
result3 = GLTBCiRDD.map(lambda x: x.split(',')) \
    .filter(lambda x: x[4] == "China" and x[0].startswith("2004")) \
    .map(lambda x: ((x[0][:4], x[3]), float(x[1]) if x[1] else float('-inf'))) \
    .filter(lambda x: x[1] < 0) \
    .map(lambda x: (x[0][0], x[0][1])) \
    .distinct() \
    .sortBy(lambda x: (x[0], x[1])) \
    .collect()

for year, city in result3:
    print(f"{year},{city}")

2004,Acheng
2004,Aksu
2004,Altay
2004,Anda
2004,Anqiu
2004,Anshan
2004,Anyang
2004,Badaojiang
2004,Baicheng
2004,Baiyin
2004,Baoding
2004,Baoji
2004,Baoshan
2004,Beian
2004,Beipiao
2004,Benxi
2004,Binzhou
2004,Boshan
2004,Cangzhou
2004,Changchun
2004,Changji
2004,Chaoyang
2004,Chengde
2004,Chifeng
2004,Dalian
2004,Dandong
2004,Datong
2004,Dawukou
2004,Dehui
2004,Dezhou
2004,Didao
2004,Dingzhou
2004,Dongling
2004,Dongying
2004,Dunhua
2004,Fengcheng
2004,Fushun
2004,Fuxin
2004,Fuyu
2004,Gaomi
2004,Gongzhuling
2004,Haibowan
2004,Haicheng
2004,Hailar
2004,Hailun
2004,Hami
2004,Handan
2004,Hangu
2004,Harbin
2004,Hebi
2004,Hegang
2004,Hengshan
2004,Hengshui
2004,Hohhot
2004,Honggang
2004,Houma
2004,Huadian
2004,Huangcun
2004,Hulan
2004,Hulan Ergi
2004,Jiamusi
2004,Jiaohe
2004,Jiaozhou
2004,Jiayuguan
2004,Jilin
2004,Jinan
2004,Jinchang
2004,Jining
2004,Jinxi
2004,Jinzhou
2004,Jishu
2004,Jiupu
2004,Jiutai
2004,Jixi
2004,Karamay
2004,Kashi
2004,Korla
2004,Kuytun
2004,Laiwu
2004,Langfang
2004,La